In [1]:
!pip install psycopg2-binary
!pip install pandas

     |████████████████████████████████| 3.4 MB 109 kB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 185 kB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 1.6 MB/s eta 0:00:01


In [1]:
import psycopg2
import pandas as pd
import db_engine as dbe

In [2]:
params = {
    "host"      : "pg_container",
    "database"  : "cargill_db",
    "user"      : "cargill",
    "password"  : "cargill"
}

In [3]:
cur, conn = dbe.create_connection(params)

Connecting to the PostgreSQL database
PostgreSQL database version:
('PostgreSQL 13.3 (Debian 13.3-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',)


In [4]:
dbe.drop_tables(cur, conn)

Executing: DROP TABLE IF EXISTS Sales
Executing: DROP TABLE IF EXISTS Discounts
Executing: DROP TABLE IF EXISTS Sales2
Executing: DROP TABLE IF EXISTS Order_detail


In [5]:
dbe.create_tables(cur, conn)

Tables created


In [6]:
dbe.insert_all(cur, conn)

Records were inserted


# Base Case

- This approach meets all the conditions that the requirement is asking
- The update of the discount applied has more complexity

![initial](https://user-images.githubusercontent.com/8701464/125843250-e545b578-dc04-41b9-a3ca-a5e22d72b060.png)

In [7]:
columns = ["sales_order_id", "sales_order_item", "customer_id", "date", "transaction_value", "discounted_value" ]
df = dbe.pg_to_pd(cur, "select * from sales", columns)
df.head()

,sales_order_id,sales_order_item,customer_id,date,transaction_value,discounted_value
0,1,2,150,2021-07-16 00:03:19.190933,200.0,200.0
1,1,3,150,2021-07-16 00:03:19.190933,310.0,310.0
2,1,4,150,2021-07-16 00:03:19.190933,80.0,80.0


In [8]:
columns = ["sales_order_id", "customer_id", "discount_value" ]
df = dbe.pg_to_pd(cur, "select * from discounts", columns)
df.head()

,sales_order_id,customer_id,discount_value
0,1,150,0.3


In [9]:
dbe.update(cur, conn, 'sales', [1, 150])

Table updated


In [10]:
columns = ["sales_order_id", "sales_order_item", "customer_id", "date", "transaction_value", "discounted_value" ]
df = dbe.pg_to_pd(cur, "select * from Sales", columns)
df.head()

,sales_order_id,sales_order_item,customer_id,date,transaction_value,discounted_value
0,1,2,150,2021-07-16 00:03:19.190933,200.0,140.0
1,1,3,150,2021-07-16 00:03:19.190933,310.0,217.0
2,1,4,150,2021-07-16 00:03:19.190933,80.0,56.0


# Edge case

- This approach meets the requirements of the problem, but also has more advanced features.
- You can have an order with more than one unit of the same item (quantity)
- Each discount can be an independent percentage
- Discount update is less complex

![second](https://user-images.githubusercontent.com/8701464/125843261-a78879e1-528b-4931-9598-d76d5d57e1f1.png)

In [11]:
columns = ["sales_order_id", "customer_id", "date"]
df = dbe.pg_to_pd(cur, "select * from Sales2", columns)
df.head()

,sales_order_id,customer_id,date
0,1,150,2021-07-16 00:03:19.200752


In [12]:
columns = ["sales_order_id", "sales_order_item", "transaction_value", "quantity", "discount_value", "discounted_value"]
df = dbe.pg_to_pd(cur, "select * from Order_detail", columns)
df.head()

,sales_order_id,sales_order_item,transaction_value,quantity,discount_value,discounted_value
0,1,2,150.0,2,0.3,150.0
1,1,3,210.0,1,0.4,210.0
2,1,4,80.0,3,0.2,80.0


In [13]:
dbe.update(cur, conn, 'order_detail', [1])

Table updated


In [14]:
columns = ["sales_order_id", "sales_order_item", "transaction_value", "quantity", "discount_value", "discounted_value"]
df = dbe.pg_to_pd(cur, "select * from Order_detail", columns)
df.head()

,sales_order_id,sales_order_item,transaction_value,quantity,discount_value,discounted_value
0,1,2,150.0,2,0.3,210.0
1,1,3,210.0,1,0.4,126.0
2,1,4,80.0,3,0.2,192.0


In [15]:
dbe.close_connection(cur, conn)

Database connection closed
